## Historical Weather Forecast data in CS
<p> https://www.worldweatheronline.com/developer/api/ </p>
<p> https://pypi.org/project/wwo-hist/ </p>


In [ ]:
# !pip install wwo-hist
# !pip install unidecode

In [ ]:
from pathlib import Path
import pandas as pd
from wwo_hist import retrieve_hist_data
from typing import List, Iterable
import os
import time
import sys
import unicodedata

In [ ]:
API_KEY_LIST = [
    "a95cd7210fbf4b85a7e185308250307",
    "5ccf4c0bfb8849ddaf8204104250207",
    "9089449cfc564000aef202420250307",
    "a486a31885364323a5a211207250307",
    "3a66d0209bd94f57a76212049250307"
    ]

API_KEY = "5ccf4c0bfb8849ddaf8204104250207"


NOTEBBOKS_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
BASES_DIR = Path.cwd().parent.parent / "Bases"
WWO_Api_DIR = BASES_DIR / "Clima"/ "worldWeatherApi"
DONE_FILE  = WWO_Api_DIR /"Temporaria"  / "done.txt"


arquivo_municipios = BASES_DIR / "Municipios" / "portugalMunicipios.csv"


# MOSTRA OS CAMINHOS
print("NOTEBBOKS_DIR:", NOTEBBOKS_DIR)
print("WWO_Api_DIR:", WWO_Api_DIR)
print("DONE_FILE:", DONE_FILE)
print("BASES_DIR:", BASES_DIR)
print("arquivo_municipios:", arquivo_municipios)


In [ ]:
def remover_acentos(texto):
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')   

In [ ]:
def get_municipios_portugal(arquivo_municipios: Path = arquivo_municipios) -> List[str]:
    """
    Lê o arquivo de municípios de Portugal e retorna uma lista de cidades.
    O arquivo deve estar no formato CSV com colunas 'Regiao' e 'Cidades'.
    """

    if not arquivo_municipios.exists():
        raise FileNotFoundError(f"Arquivo de municípios não encontrado: {arquivo_municipios}")

    # ler o arquivo de municipios em um dataframe
    # separadando os campos por vírgula
    # monte uma lista concatenando: "Portugal," [Cidades]
    import pandas as pd
    df = pd.read_csv(arquivo_municipios, sep=",", encoding="utf-8")

    #ordene por Regiao e cidades
    df = df.sort_values(by=["Regiao", "Cidades"])

    # monte uma lista concatenando: "Portugal," [Cidades]
    # cidades = [remover_acentos(cidade) + ",Portugal" for cidade in df["Cidades"].tolist()]
    cidades = [cidade + ",Portugal" for cidade in df["Cidades"].tolist()]
    
    return cidades


In [ ]:

# ------------------------------------------------------------------ utilidades
def save_lines(lines: Iterable[str], file: Path) -> None:
    file.write_text("\n".join(lines), encoding="utf-8")

def read_lines(file: Path) -> List[str]:
    return file.read_text(encoding="utf-8").splitlines() if file.exists() else []

# ------------------------------------------------------------------ download
def fetch_history(city: str, start: str, end: str, freq: int = 24, pause: float = 2):
    """Baixa os dados da cidade e grava o CSV DENTRO de BASE_DIR."""
    time.sleep(pause)

    cwd = Path.cwd()            # guarda onde eu estava
    try:
        os.chdir(WWO_Api_DIR)      # muda só durante o download
        retrieve_hist_data(
            API_KEY, [city], start, end, freq,
            location_label=False, export_csv=True, store_df=True
        )
    finally:
        os.chdir(cwd)           # volta ao normal

# ------------------------------------------------------------------ orquestra
def process_weather(
    locations: List[str],
    start_date="2025-06-25",
    end_date="2025-07-01",
    freq=24,
    retries=3,
):
    done = set(read_lines(DONE_FILE))

    for city in locations:
        if city in done:
            continue

        for attempt in range(1, retries + 1):
            try:
                fetch_history(city, start_date, end_date, freq)
                done.add(city)
                save_lines(sorted(done), DONE_FILE)
                break
            except Exception as err:
                print(f"Falhou {city} ({attempt}/{retries}): {err}")
                 # Verifica se a mensagem do erro tem "Too Many Requests"
                if "429" in str(err) or "Too Many Requests" in str(err):
                    print("Erro 429: Requisições em excesso. Encerrando o processo.")
                    sys.exit(1)  # ou raise err, se preferir deixar o erro estourar
                if attempt == retries:
                    print("Desisti dessa cidade.")


In [ ]:
municipios_portugal = get_municipios_portugal(arquivo_municipios)
municipios_portugal
# substua espacos por "+"
municipios_portugal = [cidade.replace(" ", "+") for cidade in municipios_portugal]
municipios_portugal = [remover_acentos(cidade) for cidade in municipios_portugal]
municipios_portugal

In [15]:
process_weather(
    municipios_portugal,
    start_date="2024-06-30",
    end_date="2025-06-30",
    freq=24,
    retries=3)